In [1]:
import numpy as np
import theano
import theano.tensor as T

import timeit

In [2]:
def load_data(dataset = 'gibbs-sample.dat.npy', borrow = True):
    """
    Loads the dataset.
    """
    data = np.load(dataset)
    dataset = theano.shared(np.asarray(data, dtype = theano.config.floatX), borrow = borrow)
        
    return dataset   

In [41]:
class mpf(object):
    """
    Minimum probability flow
    """
    def __init__(self, input, n, epsilon = 0.01):
        """
        Initialize the parameters of the MPF network
        """
      
        self.W = theano.shared(np.asarray(np.triu(np.random.rand(n, n), 1) + np.triu(np.random.rand(n, n), 1).T, \
                                          dtype = theano.config.floatX), name = 'W', borrow =True)
        self.b = theano.shared(value = np.zeros( n, dtype = theano.config.floatX), name = 'b', borrow = True)
        self.input = input
        self.z = T.dot(self.input, self.W) + self.b
        
        self.params = [self.W, self.b]
      
        
       
    
    def Kcost(self, epsilon = 0.01):
        """
        Returns the cost.
        Inputs:
        - data: numpy array of data that contains the different states of the network of size n
        - W: (n, n) numpy array of the weight matrix
        - b: (1, n) numpy array of biases
        - espilon: parameter for the cost
        """
        return T.mean(T.exp((0.5 - self.input) * self.z)) * epsilon

        
def sgd_opt(learning_rate = 0.01, n_epochs = 50, dataset = 'gibbs-sample.dat.npy', batch_size = 500):
    """
    Perform stochastic gradient descent on MPF.

    """
    print ('Loading the file', dataset+'...')
    
    dataset = load_data()

    n_dataset_batches = dataset.get_value(borrow = True).shape[0] // batch_size

    print ('Building the model...')

    index = T.lscalar()

    x = T.matrix('x')

    energy = mpf(input = x, n = 16)

    cost = energy.Kcost()

    grad_W = T.grad(cost = cost, wrt = energy.W)
    grad_b = T.grad(cost = cost, wrt = energy.b)

    updates = [(energy.W, energy.W - learning_rate * grad_W), (energy.b, energy.b - learning_rate * grad_b)]

    train_model = theano.function(inputs = [index],
                                outputs = cost,
                                updates = updates,
                                givens = {x: dataset[index * batch_size: (index + 1) * batch_size, :]} )

    print ('Training the model...')

    
    start_time = timeit.default_timer()

    done_looping = False
    epoch = 0
    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
#         print ('At epoch', epoch)
        for minibatch_index in range(n_dataset_batches):
            minibatch_avg_cost = train_model(minibatch_index)
            print (minibatch_avg_cost)
    
    
    
    end_time = timeit.default_timer()

    print('The code run for %d epochs, with %f epochs/sec' % (epoch, 1. * epoch / (end_time - start_time)))        

    return energy.W.get_value(), energy.b.get_value()
        
        
        
        
        
        

In [42]:
W, b = sgd_opt()

Loading the file gibbs-sample.dat.npy...
Building the model...
Grad dot.0
Training the model...
0.09220241219861267
0.09220958448899096
0.09278583142231707
0.0933229592109219
0.09184041062046089
0.09160202783686221
0.09289073453698192
0.09271014794050299
0.09141660025913605
0.09233681092314014
0.09139749271501627
0.09225273139618462
0.09053253294635567
0.09004181590567953
0.09071099671301068
0.09146441620048817
0.09031663792651477
0.08928105418966578
0.08983873502763831
0.09148492774058742
0.09015973786540137
0.09039161554120421
0.08871368197565715
0.08826545295853845
0.09033882226385778
0.09033748785377194
0.08958163081406702
0.09029025811314952
0.08967182014363553
0.08744268500798101
0.09025754103015163
0.0896774809362368
0.08774800352069853
0.08746503690338342
0.08850350318554245
0.08856460397691984
0.08804938205503327
0.08889595772336502
0.08723699926902348
0.08767046608207764
0.08805136974988573
0.08819614656866463
0.08721427906529515
0.08714573006267078
0.08704826493512605
0.0869